In [1]:
import findspark
findspark.init('/usr/local/spark/')
import pyspark

In [2]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Python Spark SQL example").getOrCreate()

In [3]:
from pyspark.sql import Row

In [4]:
sc = spark.sparkContext

In [5]:
clines = sc.textFile("/home/hduser/Downloads/sharedfolder/customers.tsv")

In [6]:
from pyspark.sql.types import *

In [7]:
cfields = clines.map(lambda l: l.split("\t"))

In [8]:
customers = cfields.map(lambda p: (p[0], p[1], p[2], p[3], p[4]))

Schema encoding in a string

In [10]:
schemaString = "cid cname ccity cstate czip"

In [11]:
columns = [StructField(column_name, StringType(),True) for column_name in schemaString.split()]

In [13]:
schema = StructType(columns)

In [14]:
columns
schema

StructType(List(StructField(cid,StringType,true),StructField(cname,StringType,true),StructField(ccity,StringType,true),StructField(cstate,StringType,true),StructField(czip,StringType,true)))

In [15]:
customerDF = spark.createDataFrame(customers, schema)

In [16]:
customerDF.printSchema()

root
 |-- cid: string (nullable = true)
 |-- cname: string (nullable = true)
 |-- ccity: string (nullable = true)
 |-- cstate: string (nullable = true)
 |-- czip: string (nullable = true)



In [17]:
customerDF.select("cname").show()

+----------------+
|           cname|
+----------------+
|     Mary Torres|
|      Jose Haley|
|      Mary Smith|
|  Richard Maddox|
|  Margaret Booth|
|  Mary Henderson|
|     Lisa Walker|
|   Jonathan Hill|
|Carolyn Sheppard|
|    Mary Mendoza|
|   Michael Smith|
|    James Holmes|
|     Mary Dawson|
|    Adam Marquez|
|    Gloria Smith|
|       Mary Webb|
|  Nancy Alvarado|
|  Russell Flores|
|    Denise Smith|
|  Jose Dickerson|
+----------------+
only showing top 20 rows



In [18]:
customerDF.select(customerDF['cname'],customerDF['ccity']).show()

+----------------+-------------+
|           cname|        ccity|
+----------------+-------------+
|     Mary Torres|       Caguas|
|      Jose Haley|     Columbus|
|      Mary Smith|      Houston|
|  Richard Maddox|       Caguas|
|  Margaret Booth|    Arlington|
|  Mary Henderson|       Caguas|
|     Lisa Walker|       Caguas|
|   Jonathan Hill|      Phoenix|
|Carolyn Sheppard|Pompano Beach|
|    Mary Mendoza|       Caguas|
|   Michael Smith|       Caguas|
|    James Holmes|     Hilliard|
|     Mary Dawson|       Caguas|
|    Adam Marquez|  San Antonio|
|    Gloria Smith|       Caguas|
|       Mary Webb|   San Marcos|
|  Nancy Alvarado|     Flushing|
|  Russell Flores|       Caguas|
|    Denise Smith|    Rego Park|
|  Jose Dickerson|         Mesa|
+----------------+-------------+
only showing top 20 rows



In [19]:
customerDF.filter(customerDF['cstate']=='CA').show()

+-----+----------------+---------------+------+-----+
|  cid|           cname|          ccity|cstate| czip|
+-----+----------------+---------------+------+-----+
| 5577|      Mary Smith|        Modesto|    CA|95350|
| 1745|      Mary Smith|Rowland Heights|    CA|91748|
|11444|Kathleen Patrick|      San Diego|    CA|92109|
| 8846|    Thomas Smith|          Indio|    CA|92201|
| 6237|  Bobby Anderson|       El Cajon|    CA|92020|
| 4085|       Mary Carr|  Panorama City|    CA|91402|
| 8705|  Patricia Smith|       Stockton|    CA|95207|
| 3669|       Mary Soto| San Bernardino|    CA|92410|
| 6101|      Mary Smith|    Los Angeles|    CA|90033|
|11697|  Jessica Thomas|  Laguna Niguel|    CA|92677|
| 1295|   Theresa Lopez|       Winnetka|    CA|91306|
| 4814|     Paul Suarez|    Simi Valley|    CA|93065|
| 8530|   William Smith|       Highland|    CA|92346|
| 3846|    Ronald Lewis|        Ontario|    CA|91764|
|10476|     John Hodges|       Cerritos|    CA|90703|
|10243|  Donna Anderson|    

In [20]:
customerDF.groupBy("cstate").count().show()

+------+-----+
|cstate|count|
+------+-----+
|    AZ|   19|
|    SC|    2|
|    LA|    7|
|    MN|    1|
|    NJ|   19|
|    DC|    4|
|    OR|    4|
|    VA|   14|
|    RI|    2|
|    KY|    1|
|    MI|   28|
|    NV|   16|
|    WI|    9|
|    ID|    2|
|    CA|  187|
|    CT|    8|
|    NC|   19|
|    MD|   19|
|    DE|    1|
|    MO|   13|
+------+-----+
only showing top 20 rows



In [21]:
customerDF.createOrReplaceTempView("customers")

In [22]:
cstate50 = spark.sql("Select cstate, count(*) as sttcount FROM customers GROUP BY cstate Having sttcount>=50")

In [23]:
cstate50.show()

+------+--------+
|cstate|sttcount|
+------+--------+
|    CA|     187|
|    NY|      79|
|    TX|      62|
|    PR|     505|
+------+--------+



In [24]:
cstate50.printSchema()

root
 |-- cstate: string (nullable = true)
 |-- sttcount: long (nullable = false)



In [25]:
type(cstate50)

pyspark.sql.dataframe.DataFrame